 # Budget Analysis

---

## Environment Setup

In [1]:
# Initial Imports
import os
import plaid
import requests
import datetime
import json
import pandas as pd
from dotenv import load_dotenv
import datetime as dt

%matplotlib inline

In [2]:
# Helper functions

def pretty_print_response(response):
    """
    This function formats a JSON file to pretty printing it in the screen.
    """
    print(json.dumps(response, indent=4, sort_keys=True))

In [3]:
# Set environment variables from the .env file
load_dotenv()

True

In [4]:
# Extract API keys from environment variables
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SECRET = os.getenv("PLAID_SECRET")
PLAID_PUBLIC_KEY = os.getenv("PLAID_PUBLIC_KEY")

---

## Plaid Access Token

In this section, you will use the `plaid-python` API to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:

1. Create a client to connect to plaid

2. Use the client to generate a public token and request the following items: ['transactions', 'income', 'assets']

3. Exchange the public token for an access token

4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [5]:
# Create the client object
client = plaid.Client(
    client_id=PLAID_CLIENT_ID,
    secret=PLAID_SECRET,
    public_key=PLAID_PUBLIC_KEY,
    environment="sandbox")

In [6]:
# Get institutions
client.Institutions.get(1)

{'institutions': [{'country_codes': ['US'],
   'credentials': [{'label': 'Username', 'name': 'username', 'type': 'text'},
    {'label': 'Password', 'name': 'password', 'type': 'password'}],
   'has_mfa': True,
   'input_spec': 'fixed',
   'institution_id': 'ins_112060',
   'mfa': ['code', 'list', 'questions', 'selections'],
   'mfa_code_type': 'numeric',
   'name': '1st Bank (Broadus, MT) - Personal',
   'oauth': False,
   'products': ['assets',
    'auth',
    'balance',
    'transactions',
    'income',
    'identity'],
   'routing_numbers': []}],
 'request_id': 'cYOxdUo5qDg3JtX',
 'total': 11391}

In [7]:
# Set the institution id
INSTITUTION_ID = "ins_109508"

### 2. Generate a public token

In [8]:
# Create the public token
create_tkn_response = client.Sandbox.public_token.create(INSTITUTION_ID, ["transactions","income","assets"])

# Print the public token
print(create_tkn_response)



{'public_token': 'public-sandbox-fbd9a8b2-1234-4dac-9670-ab92fa3327bb', 'request_id': '6SuvNER71EEegPu'}


### 3. Exchange the public token for an access token

In [9]:
# Retrieve the access token
exchange_response = client.Item.public_token.exchange(create_tkn_response["public_token"])

# Print the access toke
print(exchange_response)

{'access_token': 'access-sandbox-c6f05601-60f3-4d5b-ae2d-4fa5cc2a4600', 'item_id': 'nlAj9NxDl5CR75G3QVK4tEXXnay4bEF6BzvN3', 'request_id': 'yBusITIeNUlvCu9'}


In [10]:
# Store the access token in a Python variable
access_token = exchange_response["access_token"]

### 4. Fetch Accounts

In [11]:
# Retrieve accounts
client.Accounts.get(access_token)

# Print accounts


{'accounts': [{'account_id': 'lvANoaBDvZtQVWGzRNldivNVXAwwqMCZB1619',
   'balances': {'available': 100,
    'current': 110,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '0000',
   'name': 'Plaid Checking',
   'official_name': 'Plaid Gold Standard 0% Interest Checking',
   'subtype': 'checking',
   'type': 'depository'},
  {'account_id': 'qrAQo74DrRc879WQjznJHm514oVVb6Udl8BJw',
   'balances': {'available': 200,
    'current': 210,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '1111',
   'name': 'Plaid Saving',
   'official_name': 'Plaid Silver Standard 0.1% Interest Saving',
   'subtype': 'savings',
   'type': 'depository'},
  {'account_id': 'KqPmWM3dqnTeoxpP7G5zC1EDK7aaokFVogr6e',
   'balances': {'available': None,
    'current': 1000,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '2222',
   'name': 'Plaid CD',
   'officia

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days

2. Print the categories for each transaction type

3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)

4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [12]:
# Set the start and end date 90 days from today
start_date = "{:%Y-%m-%d}".format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = "{:%Y-%m-%d}".format(datetime.datetime.now())


# Retrieve the transactions for the last 90 days
transaction_response = client.Transactions.get(access_token,start_date,end_date)

# Print the transactions
print(json.dumps(transaction_response["transactions"][ :2],indent=4, sort_keys=True))

[
    {
        "account_id": "rzwZoeNDzJu46KGlXM3jSjXEm1PPxzFlbLNyE",
        "account_owner": null,
        "amount": 500,
        "authorized_date": null,
        "category": [
            "Travel",
            "Airlines and Aviation Services"
        ],
        "category_id": "22001000",
        "date": "2020-06-29",
        "iso_currency_code": "USD",
        "location": {
            "address": null,
            "city": null,
            "country": null,
            "lat": null,
            "lon": null,
            "postal_code": null,
            "region": null,
            "store_number": null
        },
        "merchant_name": "United Airlines",
        "name": "United Airlines",
        "payment_channel": "in store",
        "payment_meta": {
            "by_order_of": null,
            "payee": null,
            "payer": null,
            "payment_method": null,
            "payment_processor": null,
            "ppd_id": null,
            "reason": null,
            "refer

### 2. Print the categories for each transaction

In [13]:
# Create a for-loop to print the categories for each transaction
#print(json.dumps(transaction_response["transactions"][0]["category"],indent=4, sort_keys=True))


for transaction in transaction_response["transactions"]:
     print(transaction["category"])




['Travel', 'Airlines and Aviation Services']
['Travel', 'Taxi']
['Food and Drink', 'Restaurants']
['Payment']
['Food and Drink', 'Restaurants', 'Fast Food']
['Shops', 'Sporting Goods']
['Payment', 'Credit Card']
['Travel', 'Taxi']
['Transfer', 'Debit']
['Transfer', 'Deposit']
['Recreation', 'Gyms and Fitness Centers']
['Travel', 'Airlines and Aviation Services']
['Food and Drink', 'Restaurants', 'Fast Food']
['Food and Drink', 'Restaurants', 'Coffee Shop']
['Food and Drink', 'Restaurants']
['Transfer', 'Credit']
['Travel', 'Airlines and Aviation Services']
['Travel', 'Taxi']
['Food and Drink', 'Restaurants']
['Payment']
['Food and Drink', 'Restaurants', 'Fast Food']
['Shops', 'Sporting Goods']
['Payment', 'Credit Card']
['Travel', 'Taxi']
['Transfer', 'Debit']
['Transfer', 'Deposit']
['Recreation', 'Gyms and Fitness Centers']
['Travel', 'Airlines and Aviation Services']
['Food and Drink', 'Restaurants', 'Fast Food']
['Food and Drink', 'Restaurants', 'Coffee Shop']
['Food and Drink', 'R

### 3. Create a new DataFrame using the following fields from the JSON transaction data: `date`, `name`, `amount`, `category`. 

(For categories with more than one label, just use the first category label in the list)

In [14]:
# Define an empty DataFrame and set the columns as date, name, amount, and category

df = pd.DataFrame()

#columns=["date", "name", "amount", "category"]


# Retrieve all the transactions
date = []
for transaction in transaction_response["transactions"]:
    date.append(transaction["date"])
#print(date)

name = []
for transaction in transaction_response["transactions"]:
    name.append(transaction["name"])
#print(name)

amount = []
for transaction in transaction_response["transactions"]:
    amount.append(transaction["amount"])
#print(amount)

category = []
for transaction in transaction_response["transactions"]:
    category.append(transaction["category"][0])
#print(category)


# Populate the transactions DataFrame with the transactions data

df = pd.DataFrame({'date':date,'name':name, 'amount':amount,'category':category}, columns=['date','name','amount','category'])                               
                                

# Display sample data from the DataFrame   
df.head()


,date,name,amount,category
0,2020-06-29,United Airlines,500.00,Travel
1,2020-06-27,Uber 072515 SF**POOL**,6.33,Travel
2,2020-06-24,Tectra Inc,500.00,Food and Drink
3,2020-06-23,AUTOMATIC PAYMENT - THANK,2078.50,Payment
4,2020-06-23,KFC,500.00,Food and Drink


### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [15]:
# Display data types
df.dtypes

date         object
name         object
amount      float64
category     object
dtype: object

In [16]:
# Make any required data type transformation
#Format date
df['date'] = pd.to_datetime(df.date, dayfirst=True)
df.head()


,date,name,amount,category
0,2020-06-29,United Airlines,500.00,Travel
1,2020-06-27,Uber 072515 SF**POOL**,6.33,Travel
2,2020-06-24,Tectra Inc,500.00,Food and Drink
3,2020-06-23,AUTOMATIC PAYMENT - THANK,2078.50,Payment
4,2020-06-23,KFC,500.00,Food and Drink


In [17]:
# Set the date column as index
#set Index
df = df.set_index("date")

#sort date in ascending order
df = df.sort_index(ascending=True)

df.head()

# Display sample data


,name,amount,category
date,,,
2020-04-10,INTRST PYMNT,-4.22,Transfer
2020-04-11,SparkFun,89.40,Food and Drink
2020-04-12,McDonald's,12.00,Food and Drink
2020-04-12,Starbucks,4.33,Food and Drink
2020-04-13,United Airlines,-500.00,Travel


 ---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:

1. Determine the previous year's gross income and print the results

2. Determine the current monthly income and print the results

3. Determine the projected yearly income and print the results

In [18]:
# Fetch de income data

# Print the income data


In [20]:
# Set the start and end date for last year
start_date = "{:%Y-%m-%d}".format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = "{:%Y-%m-%d}".format(datetime.datetime.now())


# Retrieve the transactions for  lastyears
#transaction_response = client.Transactions.get(access_token,start_date,end_date)

# Print the transactions
#print(json.dumps(transaction_response["transactions"][ :2],indent=4, sort_keys=True))

In [ ]:
# # Set the start and end date  days for previous year

# # Set timeframe to "1D"
# timeframe = "1D"

# start_date = pd.Timestamp("2019-01-01", tz="America/New_York").isoformat()
# end_date = pd.Timestamp("2019-12-31", tz="America/New_York").isoformat()

# # Get 1 year"s worth of income
# df = api.get_barset(
#     ticker,
#     timeframe,
#     limit=None,
#     start=start_date,
#     end=end_date,
#     after=None,
#     until=None,
# ).df

# df.head()



# Print the transactions
#print(json.dumps(transaction_response["income"][ :2],indent=4, sort_keys=True))

In [ ]:
# Determine the previous year's gross income and print the results


In [ ]:
# Determine the current monthly income and print the results


In [ ]:
# Determine the projected yearly income and print the results


---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)

2. Generate a bar chart with the number of transactions for each category

3. Calulate the expenses per month

4. Plot the total expenses per month

### Calculate the expenses per category

In [24]:
# Compute the total expenses per category

# Group by `Category` and perform count
df.groupby('category').sum()


# Display the expenses per catefory


,amount
category,
Food and Drink,3317.19
Payment,6310.50
Recreation,235.50
Shops,1500.00
Transfer,20537.34
Travel,35.19


In [ ]:
# Create a spending categories pie chart.


### Calculate the expenses per month

In [ ]:
# Create a DataFrame with the total expenses

# Display sample data


In [ ]:
# Create a spending per month bar chart
